<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-12 15:03:38
-------------------
qualified stocks: 94
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.03 L
Current:  1.26 C
-------------------
Today PnL: 4.61 L (3.79%)
Current PnL: -19.34 L (-13.71%)
CY Booked + Current PnL: -16.47 L (-11.68%)
-------------------
Total profit:  3.85 L
Total loss:  -23.19 L
-------------------
Total Booked + Current PnL: 10.51 L (9.22%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.27%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 81.22 L (64.3%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.43 C (113.1%)
Deployed:  1.14 C
Current:  1.26 C
CAGR/XIRR %: 7.42%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,21.99,X-MC,88.30,140060.0,25282.0,10448.0,1.73,22.03,7.46,31.13,215.0,2.42,1.15,36.43,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,-1.02,H-LC,82.98,147034.0,21329.0,14659.0,2.98,16.97,9.97,28.63,116.0,1.46,1.20,30.99,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-11.48,X-LC,50.00,92382.0,17260.0,14901.0,3.34,22.98,16.13,42.81,87.0,1.16,0.76,15.86,X40,BTT,CHEMICALS
70,SBIN,760.30,863.00,-15.73,M-LC,60.64,212848.0,11368.0,15857.0,3.07,5.64,7.45,13.51,91.0,0.72,1.74,16.61,XY25,NTT,BANKS
25,DMART,3484.00,5112.00,-21.97,X-LC,9.57,96523.0,12907.0,26167.0,1.24,15.44,27.11,46.73,123.0,0.49,0.79,18.15,AR,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CERA,8421.60,10420.45,112.26,X-SC,93.62,84903.0,-24578.0,50560.0,11.99,-22.45,59.55,23.73,122.0,-0.49,0.69,28.50,X40N,ATH,CERAMICS
88,VALIANTORG,512.64,838.00,-294.01,H-SC,1.06,82446.0,-50840.0,135434.0,9.99,-38.14,164.27,63.47,83.0,-0.38,0.67,38.20,XR,NTT,CHEMICALS
16,BSOFT,628.85,844.81,-21.38,M-SC,87.23,73850.0,-36199.0,73990.0,9.77,-32.89,100.19,34.34,20.0,-0.49,0.60,21.58,XR,ATH,IT
65,SAIL,130.64,228.00,78.75,M-MC,57.45,203540.0,-21422.0,189068.0,8.43,-9.52,92.89,74.53,217.0,-0.11,1.66,18.18,XY24,BTT,STEEL
82,TITAGARH,1097.23,1548.00,-25.94,H-SC,7.45,129981.0,-62034.0,140912.0,8.08,-32.31,108.41,41.08,76.0,-0.44,1.06,10.01,XY24,NTT,ENGINEERING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1354.50,1800.0,-39.16,H-MC,74.47,44990.0,-32216.0,57610.0,-3.53,-41.73,128.05,32.89,41.0,-0.56,0.37,23.90,XR,NTT,BANKS
12,BANKINDIA,113.49,190.0,-28.40,M-MC,26.60,125339.0,-3813.0,90883.0,-0.07,-2.95,72.51,67.42,71.0,-0.04,1.02,21.90,XR,NTT,BANKS
55,PFIZER,4236.33,6318.3,-15.37,H-SC,39.36,131719.0,393.0,64147.0,-0.02,0.30,48.70,49.15,102.0,0.01,1.08,8.57,X40,ATH,PHARMA
67,SATIN,199.77,274.0,-15.90,H-SC,79.79,165691.0,-30283.0,103110.0,0.42,-15.45,62.23,37.16,142.0,-0.29,1.35,23.74,XY24,NTT,FINANCE
45,KANSAINER,299.63,340.0,-66.58,H-SC,41.49,234135.0,-35532.0,71856.0,0.93,-13.18,30.69,13.47,167.0,-0.49,1.91,17.19,XY24,NTT,PAINTS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TATAMOTORS,776.02,1065.00,-52.03,M-LC,95.74,191065.0,-14580.0,91157.0,1.76,-7.09,47.71,37.24,1.0,-0.16,1.56,24.36,XY24,NTT,AUTO
2,ABB,5319.37,8953.12,-38.51,H-LC,72.87,139550.0,6566.0,84274.0,2.60,4.94,60.39,68.31,2.0,0.08,1.14,13.79,AR,ATH,ELECTRICAL
24,DLF,683.35,1031.70,-26.04,H-LC,47.87,122580.0,-423.0,63129.0,7.80,-0.34,51.50,50.98,3.0,-0.01,1.00,12.00,AR,ATH,REALTY
10,AWL,342.88,485.00,-69.22,H-MC,6.38,166814.0,-50915.0,141158.0,4.00,-23.38,84.62,41.45,5.0,-0.36,1.36,11.43,XY24,NTT,FMCG
92,WHIRLPOOL,1151.42,2270.00,-35.92,M-SC,92.55,161315.0,15085.0,126971.0,3.87,10.32,78.71,97.15,7.0,0.12,1.32,37.81,X40,NTT,DURABLES


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,21.99,X-MC,88.30,140060.0,25282.0,10448.0,1.73,22.03,7.46,31.13,215.0,2.42,1.15,36.43,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,5.95,X-MC,13.83,186869.0,31434.0,39000.0,5.04,20.22,20.87,45.32,198.0,0.81,1.53,27.15,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-11.48,X-LC,50.00,92382.0,17260.0,14901.0,3.34,22.98,16.13,42.81,87.0,1.16,0.76,15.86,X40,BTT,CHEMICALS
57,POLYCAB,5005.92,7541.35,18.48,H-MC,90.43,168210.0,28044.0,42944.0,4.18,20.01,25.53,50.65,54.0,0.65,1.38,28.38,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,5.78,M-LC,84.04,186512.0,35306.0,30364.0,4.22,23.35,16.28,43.43,37.0,1.16,1.53,29.28,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,POLYCAB,5005.92,7541.35,18.48,H-MC,90.43,168210.0,28044.0,42944.0,4.18,20.01,25.53,50.65,54.0,0.65,1.38,28.38,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,5.78,M-LC,84.04,186512.0,35306.0,30364.0,4.22,23.35,16.28,43.43,37.0,1.16,1.53,29.28,XY24,NTT,MISC
6,ANGELONE,2328.67,3033.00,-1.24,H-SC,72.87,161518.0,10154.0,35631.0,6.81,6.71,22.06,30.25,131.0,0.28,1.32,27.84,X40N,NTT,FINANCE
18,CAMS,3643.00,5250.99,-6.99,H-SC,10.64,102757.0,753.0,44268.0,5.47,0.74,43.08,44.14,47.0,0.02,0.84,19.13,XR,ATH,MISC
93,WIPRO,243.46,420.00,-6.99,M-LC,61.70,159439.0,8494.0,100957.0,6.26,5.63,63.32,72.51,58.0,0.08,1.30,20.04,XR,NTT,IT


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
93,WIPRO,243.46,420.00,-6.99,M-LC,61.70,159439.0,8494.0,100957.0,6.26,5.63,63.32,72.51,58.0,0.08,1.30,20.04,XR,NTT,IT
18,CAMS,3643.00,5250.99,-6.99,H-SC,10.64,102757.0,753.0,44268.0,5.47,0.74,43.08,44.14,47.0,0.02,0.84,19.13,XR,ATH,MISC
37,INDIAMART,2311.97,4911.36,-53.33,H-SC,75.53,100874.0,-853.0,115228.0,1.64,-0.84,114.23,112.43,NaN,-0.01,0.82,19.06,AR,ATH,MISC
91,VOLTAS,1272.00,1929.20,-18.29,H-MC,22.87,139084.0,-836.0,73130.0,2.33,-0.60,52.58,51.67,19.0,-0.01,1.14,5.05,AR,ATH,AC
12,BANKINDIA,113.49,190.00,-28.40,M-MC,26.60,125339.0,-3813.0,90883.0,-0.07,-2.95,72.51,67.42,71.0,-0.04,1.02,21.90,XR,NTT,BANKS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UJJIVANSFB,52.77,60.00,-40.07,M-SC,77.66,115803.0,-26676.0,46194.0,5.30,-18.72,39.89,13.70,219.0,-0.58,0.95,36.68,X40N,NTT,BANKS
53,MRF,114777.60,150504.14,21.99,X-MC,88.30,140060.0,25282.0,10448.0,1.73,22.03,7.46,31.13,215.0,2.42,1.15,36.43,AR,ATH,TYRES
86,UNIONBANK,123.87,163.00,-8.12,M-LC,58.51,145593.0,4753.0,39732.0,4.19,3.37,27.29,31.59,83.0,0.12,1.19,26.47,XY24,NTT,BANKS
69,SBILIFE,1496.49,1924.99,-1.02,H-LC,82.98,147034.0,21329.0,14659.0,2.98,16.97,9.97,28.63,116.0,1.46,1.20,30.99,AR,ATH,INSURANCE
52,MEDANTA,1022.60,1486.00,5.95,X-MC,13.83,186869.0,31434.0,39000.0,5.04,20.22,20.87,45.32,198.0,0.81,1.53,27.15,XY24,NTT,HEALTHCARE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VIPIND,488.80,718.0,-799.62,H-SC,100.00,66690.0,-28626.0,73319.0,3.50,-30.03,109.94,46.89,51.0,-0.39,0.55,31.54,X40N,NTT,MISC
79,TATAELXSI,7332.28,9000.0,-18.84,X-MC,98.94,85197.0,-17455.0,40801.0,6.06,-17.00,47.89,22.74,33.0,-0.43,0.70,28.06,X40N,NTT,IT
1,AARTIIND,487.04,919.0,24.30,M-SC,97.87,78565.0,-2771.0,74904.0,4.17,-3.41,95.34,88.69,118.0,-0.04,0.64,32.48,XR,NTT,CHEMICALS
58,QUESS,711.66,986.0,NaN,NaN,96.81,77033.0,-78821.0,138898.0,6.85,-50.57,180.31,38.55,NaN,-0.57,0.63,17.96,XY24,NTT,MISC
80,TATAMOTORS,776.02,1065.0,-52.03,M-LC,95.74,191065.0,-14580.0,91157.0,1.76,-7.09,47.71,37.24,1.0,-0.16,1.56,24.36,XY24,NTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.54
1,25,41.35
2,50,69.62


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.59
X40     14.64
X40N    14.00
AR      13.78
XY25    10.35
XR      10.27
X200     1.22
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.04
X-LC    18.57
M-SC    16.44
M-LC     8.47
X-MC     6.82
H-LC     6.78
H-MC     6.78
M-MC     5.50
X-SC     2.80
L-SC     2.24
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.91,-1.78,34.15
BANKS,11.05,-11.49,66.17
IT,9.36,-20.93,88.40
FINANCE,7.33,-23.57,79.34
MISC,7.15,-29.60,85.36
INSURANCE,6.02,-4.85,36.46
PAINTS,5.63,-19.42,54.76
ELECTRICAL,4.09,-11.06,67.65
REFINERIES,3.71,10.23,6.31


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3118110.0,27
XR,1286316.0,15
AR,1268851.0,15
X40N,1221171.0,17
X40,804750.0,12
SR,208989.0,2
XY25,144542.0,5
X200,69164.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2494015.0,22
M-SC,1992327.0,20
X-LC,712141.0,12
M-MC,593044.0,5
H-MC,400404.0,6
X-SC,365487.0,4
M-LC,344389.0,6
L-SC,322743.0,4
X-MC,321904.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      947361.0      6
M-SC       XY24      798444.0      6
H-SC       X40N      569958.0      7
           AR        560208.0      3
M-SC       XR        435768.0      5
M-MC       XY24      417174.0      3
M-SC       X40N      379001.0      5
X-LC       X40       364966.0      5
H-SC       XR        311490.0      4
M-LC       XY24      227575.0      4
M-SC       X40       200130.0      2
H-LC       AR        193824.0      4
X-LC       AR        186771.0      3
H-MC       XY24      184310.0      2
L-SC       XR        165011.0      2
X-MC       XY24      140690.0      2
X-SC       XR        124597.0      1
           SR        122559.0      1
           X40N      118331.0      2
X-MC       X40N      110937.0      2
H-SC       X40       104998.0      2
M-LC       XR        100957.0      1
M-SC       AR         92554.0      1
X-LC       XY25       91240.0      3
M-MC       XR         90883.0      1
L-SC       XY24       90803.0      1
M-SC       SR         86430.0      1
M-MC       AR         84987.0      1
H-MC       AR         73130.0      1
X-LC       X200       69164.0      1
L-SC       AR         66929.0      1
X-MC       X40        59829.0      1
H-MC       XR         57610.0      1
           X40N       42944.0      1
           X40        42410.0      1
H-LC       XY25       37445.0      1
           X40        32417.0      1
M-LC       XY25       15857.0      1
X-MC       AR         10448.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
